In [1]:
from bs4 import BeautifulSoup,ResultSet
import requests
import pandas as pd
import numpy as np
import re

In [2]:
# The webpage URL
URL = "https://www.lechocolat-alainducasse.com/uk/"

In [3]:
def get_webpage(url):
    HEADERS = ({'Accept-Language': 'en-US, en;q=0.5'})
    #print(url)
    webpage = requests.get(url,headers=HEADERS)
    print(webpage)
    return webpage
    

In [4]:
def get_links(webpage,class_name):
    try:
        soup = BeautifulSoup(webpage.content, "html.parser")
        #print(soup.get_text())
        links = soup.find_all("a", attrs={'class':class_name})
        links_list = []
        for link in links:
            item_link = link.get('href')
            if URL not in item_link:
                links_list.append(URL+item_link)
            else:
                links_list.append(item_link)
    except:
        print('B')
        links_list=[]
    return links_list

In [5]:
webpage = get_webpage(URL)

<Response [200]>


In [6]:

category_links = get_links(webpage,"siteMenuItem__link")

In [7]:
category_links

['https://www.lechocolat-alainducasse.com/uk/easter-chocolate',
 'https://www.lechocolat-alainducasse.com/uk/chocolates',
 'https://www.lechocolat-alainducasse.com/uk/chocolate-bar',
 'https://www.lechocolat-alainducasse.com/uk/chocolate-gift',
 'https://www.lechocolat-alainducasse.com/uk/simple-pleasures',
 'https://www.lechocolat-alainducasse.com/uk/breakfast-snacks']

In [8]:
product_links=[]
for link in category_links:
    temp_links=get_links(get_webpage(link),"productMiniature__name")
    product_links.extend(temp_links)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [9]:
len(product_links)

151

In [10]:
product_links

['https://www.lechocolat-alainducasse.com/uk/the-hide-and-seek-box',
 'https://www.lechocolat-alainducasse.com/uk/oeuf-facette',
 'https://www.lechocolat-alainducasse.com/uk/hexa-milk-chocolate-woodpecke',
 'https://www.lechocolat-alainducasse.com/uk/chick-milk-chocolate-bar',
 'https://www.lechocolat-alainducasse.com/uk/the-dark-chocolate-chick',
 'https://www.lechocolat-alainducasse.com/uk/peacock-milk-chocolate-box',
 'https://www.lechocolat-alainducasse.com/uk/the-milk-chocolate-chick',
 'https://www.lechocolat-alainducasse.com/uk/chick-dark-chocolate-bar',
 'https://www.lechocolat-alainducasse.com/uk/hexa-dark-chocolate-woodpecker',
 'https://www.lechocolat-alainducasse.com/uk/easter-treats-milk#/77-size-150g',
 'https://www.lechocolat-alainducasse.com/uk/the-chocolate-lamb',
 'https://www.lechocolat-alainducasse.com/uk/easter-french-hen-small-dark',
 'https://www.lechocolat-alainducasse.com/uk/easter-egg-dark-t1#/77-size-150g',
 'https://www.lechocolat-alainducasse.com/uk/easter-

In [11]:
def get_title(soup):
    try:
        titl = soup.find('h1',attrs={'class': "productCard__title"}).get_text().strip().split("\n")
        title=""
        for t in titl:
            title+=t.strip()+" - "
        title=title[:-3]
    except:
        title = ""
    return title

In [12]:
def get_subtitle(soup):
    try:
        subtitle = soup.find('h2',attrs={'class': "productCard__subtitle"}).get_text().strip()
    except:
        subtitle = ""
    return subtitle

In [13]:
def get_weight(soup):
    try:
        weight = soup.find('p',attrs={'class': "productCard__weight"}).get_text().strip()
    except:
        weight = ""
    return weight

In [14]:
def get_description(soup):
    try:
        description = soup.find('div',attrs={'class': "productDescription__text wysiwyg-content product-description"}).get_text().strip()
    except:
        description = ""
    return description

In [15]:
def get_consume_advice(soup):
    try:
        consume_advice = soup.find('p',attrs={'class': "consumeAdvices"}).get_text().strip()
    except:
        consume_advice = ""
    return consume_advice

In [16]:
#productActions__addToCart button add-to-cart add
def get_price(soup):
    try:
        price = soup.find('button',attrs={'class': "productActions__addToCart button add-to-cart add"}).get_text().strip().split('£')[-1]
        price='£'+price
    except:
        price = ""
    return price


In [17]:
def get_images(soup):
    try:
        image = soup.find_all('img', attrs={'class':"productImages__image"})
        images=[]
        for img in image:
            images.append(img.get('src'))
    except:
        images=[]
    return images

In [42]:
def get_product_details(soup):
    try:
        details = soup.find_all('h3',attrs={'class': "wysiwyg-title-default"})
        info_pairs={}
        for det in details:
            info_pairs[det.get_text().lower()]=det.find_next().get_text().strip()
    except:
        info_pairs = {}
    return info_pairs

In [19]:
def get_product_availability(soup):
    try:
        availability = soup.find('div',attrs={'class': "productWarning__message"}).find_next().get_text().strip()
    except:
        availability = ""
    if availability=="":
        availability="Available in whole UK"
    return availability

In [20]:
def get_category(soup):
    #productInformation__wrapper
    try:
        category = soup.find('div',attrs={'class': "productInformation__wrapper"}).find_all('span')[1].get_text()
    except:
        category = ""
    return category

In [48]:
d = {"product_id":[],"title":[],"subtitle":[],"description":[],"category":[],"url":[], "price":[],
     "weight":[],"images":[], "consume_advice":[], "ingredients":[],"nutritional values":[],
     "allergens":[],"vegan":[],"price per kilo":[],"product_availability":[]}

In [49]:
for link in product_links:
    new_webpage=get_webpage(link)
    new_soup = BeautifulSoup(new_webpage.content, "html.parser")
    d['product_id'].append(link.split("/")[-1])
    d['title'].append(get_title(new_soup))
    d['subtitle'].append(get_subtitle(new_soup))
    d['description'].append(get_description(new_soup))
    d['category'].append(get_category(new_soup))
    d['url'].append(link)
    d['price'].append(get_price(new_soup))
    d['weight'].append(get_weight(new_soup))
    d['images'].append(get_images(new_soup))
    d['consume_advice'].append(get_consume_advice(new_soup))
    product_details = get_product_details(new_soup)
    cols=["ingredients","nutritional values","allergens","vegan","price per kilo"]
    for col in cols:
        if col in product_details:
            d[col].append(product_details[col])
        else:
            d[col].append("") 
    d['product_availability'].append(get_product_availability(new_soup))

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [51]:
lechocolat = pd.DataFrame.from_dict(d)
lechocolat = lechocolat.dropna(subset=['title'])
lechocolat.to_csv("output/lechocolat_data.csv", header=True, index=False)

In [52]:
lechocolat.shape

(151, 16)